# **Exercise 2: Data Warehousing and Data Lakes with Spark + Hive**

## Introduction

In modern data engineering, we often encounter two primary paradigms:
1. **Data Warehouse** (schema-on-write): where data is cleansed, transformed, and loaded into structured tables before analysis.
2. **Data Lake** (schema-on-read): where data is stored in raw format and the schema is applied when querying.

This exercise showcases both approaches using **Spark** and **Hive**. You will load and query **e-commerce data** in a structured (warehouse) format, then contrast this with a more flexible (lake) approach. By the end, you should understand key **ETL/ELT** concepts, the rationale behind each paradigm, and be able to discuss the differences.

**Useful links and notebooks:**
- https://spark.apache.org/docs/latest/api/python/index.html
- https://spark.apache.org/docs/3.5.1/sql-data-sources-hive-tables.html
- /shared/ETL_ELT

---

## Objectives


This exercise is worth 18 points. To earn full points, make sure to include comments in your code explaining your approach and the reasoning behind your choices.

1. **Data Warehouse Fundamentals (6p)**:  
   - Define and create schemas using Apache Hive.
   - Perform schema-on-write transformations and run analytical queries.
  
     
---
2. **Data Lake Fundamentals (6p)**:  
   - Ingest raw files into Spark without predefined schema (schema-on-read).  
   - Handle multiple file formats (CSV, JSON, TXT) in a flexible manner.

---

3. **Questions (6p)**:
   - Answer three questions about the ETL and ELT.

# E-Commerce Data Schema

We will be working with a couple of datasets from an e-commerce site located in the /shared folder.

## 1. `customers.csv`
- **Description**: Contains information about customers.
- **Fields**:
  - `customer_id` (int): Unique identifier for each customer.
  - `name` (string): Customer's full name.
  - `age` (int): Age of the customer.
  - `country` (string): Country of residence.
  - `preferred_category` (string): Preferred product category (e.g., Electronics, Books).
  - `loyalty_score` (float): Loyalty score between 0.00 and 1.00.

---

## 2. `products.csv`
- **Description**: Contains information about products.
- **Fields**:
  - `product_id` (int): Unique identifier for each product.
  - `product_name` (string): Name of the product.
  - `category` (string): Product category (e.g., Electronics, Clothing).
  - `price` (float): Unit price of the product.
  - `popularity` (int): Popularity score (1–10).
  - `region` (string): Shipping region for the product (e.g., North America, Europe).

---

## 3. `transactions.json`
- **Description**: Contains information about transactions.
- **Fields**:
  - `transaction_id` (int): Unique identifier for each transaction.
  - `customer_id` (int): ID referencing a row in `customers.csv`.
  - `product_id` (int): ID referencing a row in `products.csv`.
  - `quantity` (int): Number of items purchased in the transaction.
  - `price` (float): Unit price of the product.
  - `shipping_cost` (float): Shipping cost for the transaction.
  - `tax` (float): Tax amount applied to the transaction.
  - `total_amount` (float): Computed total cost (`quantity * price + shipping_cost + tax`).
  - `transaction_time` (string, ISO format): Timestamp of the transaction (e.g., `YYYY-MM-DDTHH:MM:SS`).

---

## 4. `reviews.txt`
- **Description**: Semi-structured text file containing product reviews.
- **Format**: Each line follows the format: `customer_id|product_id|product_name|review_text|rating`.
- **Fields**:
  - `customer_id` (int): ID referencing a row in `customers.csv`.
  - `product_id` (int): ID referencing a row in `products.csv`.
  - `product_name` (string): Name of the reviewed product.
  - `review_text` (string): Freeform text describing the customer’s opinion.
  - `rating` (int): Numeric score (1–5).

---

## Notes
- **Relationships**:
  - `customer_id` links `transactions.json` and `reviews.txt` to `customers.csv`.
  - `product_id` links `transactions.json` and `reviews.txt` to `products.csv`.



**Start by setting up a Spark session, enable Hive support so we can create databases and tables.**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Exercise2") \
    .config("spark.sql.warehouse.dir", "data_warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

spark

---
# **1. ETL: Load data into a Data Warehouse (6p)**

## Instructions
1. Define the following tables:
   - **`customers`**
   - **`products`**
   - **`transactions`**
   - **`reviews`**
2. Use **Parquet** format for optimized storage and query performance.
3. Write `CREATE TABLE` statements in Hive to define the schema.
4. **Optional**: Consider partitioning tables if you think it's reasonable, and explain the reasoning behind your decision.


In [2]:
# Creating a database to store tables
spark.sql("CREATE DATABASE IF NOT EXISTS ecommerce")
spark.sql("USE ecommerce")

DataFrame[]

In [3]:
print("Databases in Spark:")
spark.sql("SHOW DATABASES").show()

Databases in Spark:
+---------+
|namespace|
+---------+
|  default|
|ecommerce|
+---------+



In [4]:
# Customers table - Partitioned by country because it is useful for queries with country as a filter
spark.sql("""
CREATE TABLE IF NOT EXISTS ecommerce.customers (
    customer_id INT,
    name STRING,
    age INT,
    country STRING,
    preferred_category STRING,
    loyalty_score FLOAT
)
USING PARQUET
PARTITIONED BY (country)
""")

DataFrame[]

In [5]:
# Products table - Partitioned by region because it is useful for queries with regions as a filter (customers from Europe will have Europe as a filter)
spark.sql("""
CREATE TABLE IF NOT EXISTS ecommerce.products (
    product_id INT,
    product_name STRING,
    category STRING,
    price FLOAT,
    popularity INT,
    region STRING
)
USING PARQUET
PARTITIONED BY (region)
""")

DataFrame[]

In [6]:
# Transactions table - improves query performance for time-based analysis, which is the most common analysis for transactions
spark.sql("""
CREATE TABLE IF NOT EXISTS ecommerce.transactions (
    transaction_id INT,
    customer_id INT,
    product_id INT,
    quantity INT,
    price FLOAT,
    shipping_cost FLOAT,
    tax FLOAT,
    total_amount FLOAT,
    transaction_time TIMESTAMP
)
USING PARQUET
PARTITIONED BY (transaction_time)
""")

DataFrame[]

In [7]:
# Reviews table - improves speed for analysis of reviews by rating
spark.sql("""
CREATE TABLE IF NOT EXISTS ecommerce.reviews (
    customer_id INT,
    product_id INT,
    product_name STRING,
    review_text STRING,
    rating INT
)
USING PARQUET
PARTITIONED BY (rating)
""")

DataFrame[]

### **ETL Process**

Now that we have defined the tables we can extract raw data, clean it, and load it into the predefined tables.

### Instructions
1. Read raw data from the provided files located in the shared folder (`customers.csv`, `products.csv`, `transactions.json`, `reviews.txt`).
2. Apply transformations:
   - Cast columns to the correct data types.
   - Handle missing or invalid data (e.g., filter out rows with null IDs, if such rows exist)
   - Only insert the columns you find necessary.
3. Use spark.sql or DataFrame APIs to insert the cleaned data into the warehouse tables.



In [8]:
# Load raw data from shared folder
customers_df = spark.read.csv("customers.csv", header=True, inferSchema=True)
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)
transactions_df = spark.read.json("transactions.json")
reviews_rdd = spark.sparkContext.textFile("reviews.txt")

In [9]:
from pyspark.sql.functions import col, when

In [10]:
# Clean customers data
# All columns kept
cleaned_customers_df = (
    customers_df
    .filter(col("customer_id").isNotNull())  # Filter out rows with null customer_id
    .select(
        col("customer_id").cast("int"),
        col("name").cast("string"),
        col("age").cast("int"),
        col("country").cast("string"),
        col("preferred_category").cast("string"),
        col("loyalty_score").cast("float")
    )
)

In [11]:
# Clean products data
# All columns kept
cleaned_products_df = (
    products_df
    .filter(col("product_id").isNotNull())  # Filter out rows with null product_id
    .select(
        col("product_id").cast("int"),
        col("product_name").cast("string"),
        col("category").cast("string"),
        col("price").cast("float"),
        col("popularity").cast("int"),
        col("region").cast("string")
    )
)

In [12]:
# Clean transactions data
# All columns kept
cleaned_transactions_df = (
    transactions_df
    .filter(col("transaction_id").isNotNull())  # Filter out rows with null transaction_id
    .select(
        col("transaction_id").cast("int"),
        col("customer_id").cast("int"),
        col("product_id").cast("int"),
        col("quantity").cast("int"),
        col("price").cast("float"),
        col("shipping_cost").cast("float"),
        col("tax").cast("float"),
        col("total_amount").cast("float"),
        col("transaction_time").cast("timestamp")
    )
)

In [13]:
# Parse reviews data from RDD
reviews_df = reviews_rdd.map(lambda line: line.split("|")).toDF([
    "customer_id", "product_id", "product_name", "review_text", "rating"
])

# Clean reviews data
# All columns kept
cleaned_reviews_df = (
    reviews_df
    .filter(col("customer_id").isNotNull() & col("product_id").isNotNull())  # Filter out rows with null IDs
    .select(
        col("customer_id").cast("int"),
        col("product_id").cast("int"),
        col("product_name").cast("string"),
        col("review_text").cast("string"),
        col("rating").cast("int")
    )
)

In [14]:
# Write data to warehouse tables
cleaned_customers_df.write.mode("overwrite").insertInto("ecommerce.customers")
cleaned_products_df.write.mode("overwrite").insertInto("ecommerce.products")
cleaned_transactions_df.write.mode("overwrite").insertInto("ecommerce.transactions")
cleaned_reviews_df.write.mode("overwrite").insertInto("ecommerce.reviews")

In [15]:
# Check table contents
spark.sql("SELECT * FROM ecommerce.customers LIMIT 10").show()
spark.sql("SELECT * FROM ecommerce.products LIMIT 10").show()
spark.sql("SELECT * FROM ecommerce.transactions LIMIT 10").show()
spark.sql("SELECT * FROM ecommerce.reviews LIMIT 10").show()

+-----------+-----------------+---+--------------+------------------+-------------+
|customer_id|             name|age|       country|preferred_category|loyalty_score|
+-----------+-----------------+---+--------------+------------------+-------------+
|          1|    Cindy Simpson| 60|United Kingdom|          Clothing|         0.15|
|          2|       Eric White| 41|United Kingdom|          Clothing|         0.22|
|          3|       Linda Todd| 54|United Kingdom|              Home|          0.5|
|          4|    Shannon Woods| 52|        Canada|            Sports|         0.71|
|          5|    Michael Brown| 48|        France|          Clothing|         0.36|
|          6|Priscilla Stewart| 72|        France|             Books|         0.06|
|          7|    Katie Allison| 24|United Kingdom|       Electronics|         0.04|
|          8|     Jeremy Weiss| 73| United States|             Books|         0.11|
|          9| Shelly Castaneda| 59|        Canada|            Sports|       

## Analyze the Data

## Objective
Run SQL queries to analyze the transformed data.

### Example Queries to Run

1. **Total Revenue and Transactions per Product Category**  
  

2. **Identify the 5 Least Sold Products**  
  

3. **Identify the Top 5 Spending Customers**  


**You are encouraged to run these queries, but feel free to explore the data and create your own queries if you believe they provide better insights or are more relevant for analysis.**


In [18]:
# Total Revenue and Transactions per Product Category
spark.sql("""
    SELECT p.category, SUM(t.price) as total_revenue, COUNT(transaction_id) as num_of_transactions
    FROM transactions t
    JOIN products p
    ON t.product_id = p.product_id
    GROUP BY p.category
    ORDER BY num_of_transactions DESC
""").show()

+-----------+------------------+-------------------+
|   category|     total_revenue|num_of_transactions|
+-----------+------------------+-------------------+
|   Clothing|21233.000051498413|                300|
|      Books| 6439.710102081299|                229|
|Electronics| 83316.10888671875|                189|
|     Sports|31397.290817260742|                171|
|       Home|27461.890014648438|                111|
+-----------+------------------+-------------------+



In [20]:
#Identify the 5 Least Sold Products
spark.sql("""
    SELECT p.product_name, SUM(t.quantity) as total_quantity_sold
    FROM transactions t
    JOIN products p
    ON t.product_id = p.product_id
    GROUP BY p.product_name
    ORDER BY total_quantity_sold ASC
    LIMIT 5
""").show()

+--------------------+-------------------+
|        product_name|total_quantity_sold|
+--------------------+-------------------+
|        Air Purifier|                  5|
|          Wall Clock|                  5|
|        Coffee Maker|                  5|
|       Action Camera|                  7|
|Noise-Canceling H...|                  9|
+--------------------+-------------------+



In [21]:
#Identify the Top 5 Spending Customers
spark.sql("""
    SELECT c.name, SUM(t.total_amount) as total_money_spent
    FROM transactions t
    JOIN customers c
    ON t.customer_id = c.customer_id
    GROUP BY c.name
    ORDER BY total_money_spent DESC
    LIMIT 5
""").show()

+----------------+------------------+
|            name| total_money_spent|
+----------------+------------------+
|     Nancy Jones|   15662.419921875|
|     Cesar Davis| 14997.11003112793|
|Valerie Mitchell|14160.320007324219|
|  Anthony Pruitt|12767.850021362305|
|  Nicholas Davis|11635.649841308594|
+----------------+------------------+



In [43]:
# Product categories ranked by ratings
spark.sql("""
    SELECT p.category, AVG(r.rating) as avg_rating
    FROM reviews r
    JOIN products p
    ON r.product_id = p.product_id
    GROUP BY p.category
    ORDER BY avg_rating DESC
""").show()

+-----------+------------------+
|   category|        avg_rating|
+-----------+------------------+
|Electronics| 4.194444444444445|
|     Sports|3.6315789473684212|
|       Home|3.5813953488372094|
|   Clothing|3.1463414634146343|
|      Books| 3.119047619047619|
+-----------+------------------+



---
# 2. **ELT: Load Raw Data into a Data Lake (6p)**

## Objective
Copy the raw data files into a `data_lake/` directory and transform the data on read.

### Instructions
1. Copy or use shell commands or scripts to move the files into a `data_lake/` directory in your `my-work` folder.
2. Do not modify the files; load them “as is” to retain their raw state.

Now the `data_lake/` folder contains all raw files, unmodified:

```plaintext
data_lake/
├── customers.csv
├── products.csv
├── transactions.json
└── reviews.txt


In [22]:
mkdir -p data_lake

In [23]:
import shutil
import os

# Paths
source_dir = "./"
destination_dir = "data_lake/"

# Ensure destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# List of files to copy
files = ["customers.csv", "products.csv", "transactions.json", "reviews.txt"]

# Copy files
for file in files:
    shutil.copy(os.path.join(source_dir, file), destination_dir)

print("Files copied to data_lake/ successfully!")

Files copied to data_lake/ successfully!


In [24]:
ls -l data_lake/

total 216
-rw-r--r--. 1 jovyan root   4430 Jan 28 17:19 customers.csv
-rw-r--r--. 1 jovyan root   2235 Jan 28 17:19 products.csv
-rw-r--r--. 1 jovyan root  20147 Jan 28 17:19 reviews.txt
-rw-r--r--. 1 jovyan root 185370 Jan 28 17:19 transactions.json


# **Transform and Analyze**

### Instructions
1. Read the raw files from the `data_lake/` directory using Spark.
2. Clean and transform the data on read.
3. Register the transformed DataFrames as temporary views.
4. Run the same queries as in the warehouse approach:
   
- Total Revenue and Transactions per Product Category
  
- Identify the 5 Least Sold Products
  
- Identify the Top 5 Customers by Spending


**You are encouraged to run these queries, but feel free to explore the data and create your own queries if you believe they provide better insights or are more relevant for analysis.**


In [27]:
# Load raw data from data_lake folder
customers_df = spark.read.csv("data_lake/customers.csv", header=True, inferSchema=True)
products_df = spark.read.csv("data_lake/products.csv", header=True, inferSchema=True)
transactions_df = spark.read.json("data_lake/transactions.json")
reviews_rdd = spark.sparkContext.textFile("data_lake/reviews.txt")

In [28]:
# Clean customers data
cleaned_customers_df = (
    customers_df
    .filter(col("customer_id").isNotNull())  # Filter out rows with null customer_id
    .select(
        col("customer_id").cast("int"),
        col("name").cast("string"),
        col("age").cast("int"),
        col("country").cast("string"),
        col("preferred_category").cast("string"),
        col("loyalty_score").cast("float")
    )
)

In [29]:
# Clean products data
cleaned_products_df = (
    products_df
    .filter(col("product_id").isNotNull())  # Filter out rows with null product_id
    .select(
        col("product_id").cast("int"),
        col("product_name").cast("string"),
        col("category").cast("string"),
        col("price").cast("float"),
        col("popularity").cast("int"),
        col("region").cast("string")
    )
)

In [30]:
# Clean transactions data
cleaned_transactions_df = (
    transactions_df
    .filter(col("transaction_id").isNotNull())  # Filter out rows with null transaction_id
    .select(
        col("transaction_id").cast("int"),
        col("customer_id").cast("int"),
        col("product_id").cast("int"),
        col("quantity").cast("int"),
        col("price").cast("float"),
        col("shipping_cost").cast("float"),
        col("tax").cast("float"),
        col("total_amount").cast("float"),
        col("transaction_time").cast("timestamp")
    )
)

In [31]:
# Parse reviews data from RDD
reviews_df = reviews_rdd.map(lambda line: line.split("|")).toDF([
    "customer_id", "product_id", "product_name", "review_text", "rating"
])

# Clean reviews data
cleaned_reviews_df = (
    reviews_df
    .filter(col("customer_id").isNotNull() & col("product_id").isNotNull())  # Filter out rows with null IDs
    .select(
        col("customer_id").cast("int"),
        col("product_id").cast("int"),
        col("product_name").cast("string"),
        col("review_text").cast("string"),
        col("rating").cast("int")
    )
)

In [32]:
# Temporary views
cleaned_customers_df.createOrReplaceTempView("customers_view")
cleaned_products_df.createOrReplaceTempView("products_view")
cleaned_transactions_df.createOrReplaceTempView("transactions_view")
cleaned_reviews_df.createOrReplaceTempView("reviews_view")

In [33]:
#Total Revenue and Transactions per Product Category
spark.sql("""
    SELECT p.category, SUM(t.price) as total_revenue, COUNT(transaction_id) as num_of_transactions
    FROM transactions_view t
    JOIN products_view p
    ON t.product_id = p.product_id
    GROUP BY p.category
    ORDER BY num_of_transactions DESC
""").show()

+-----------+------------------+-------------------+
|   category|     total_revenue|num_of_transactions|
+-----------+------------------+-------------------+
|   Clothing|21233.000051498413|                300|
|      Books| 6439.710102081299|                229|
|Electronics| 83316.10888671875|                189|
|     Sports|31397.290817260742|                171|
|       Home|27461.890014648438|                111|
+-----------+------------------+-------------------+



In [34]:
#Identify the 5 Least Sold Products
spark.sql("""
    SELECT p.product_name, SUM(t.quantity) as total_quantity_sold
    FROM transactions_view t
    JOIN products_view p
    ON t.product_id = p.product_id
    GROUP BY p.product_name
    ORDER BY total_quantity_sold ASC
    LIMIT 5
""").show()

+--------------------+-------------------+
|        product_name|total_quantity_sold|
+--------------------+-------------------+
|        Air Purifier|                  5|
|          Wall Clock|                  5|
|        Coffee Maker|                  5|
|       Action Camera|                  7|
|Noise-Canceling H...|                  9|
+--------------------+-------------------+



In [35]:
#Identify the Top 5 Customers by Spending
spark.sql("""
    SELECT c.name, SUM(t.total_amount) as total_money_spent
    FROM transactions_view t
    JOIN customers_view c
    ON t.customer_id = c.customer_id
    GROUP BY c.name
    ORDER BY total_money_spent DESC
    LIMIT 5
""").show()

+----------------+------------------+
|            name| total_money_spent|
+----------------+------------------+
|     Nancy Jones|   15662.419921875|
|     Cesar Davis| 14997.11003112793|
|Valerie Mitchell|14160.320007324219|
|  Anthony Pruitt|12767.850021362305|
|  Nicholas Davis|11635.649841308594|
+----------------+------------------+



In [36]:
# Product categories ranked by ratings
spark.sql("""
    SELECT p.category, AVG(r.rating) as avg_rating
    FROM reviews_view r
    JOIN products_view p
    ON r.product_id = p.product_id
    GROUP BY p.category
    ORDER BY avg_rating DESC
""").show()

+-----------+------------------+
|   category|        avg_rating|
+-----------+------------------+
|Electronics| 4.194444444444445|
|     Sports|3.6315789473684212|
|       Home|3.5813953488372094|
|   Clothing|3.1463414634146343|
|      Books| 3.119047619047619|
+-----------+------------------+



---
### **Questions (6p)**

Reflect on the following questions and provide thoughtful answers. Focus on your reasoning, insights, and key takeaways from the exercise.


**1. What were the key differences in how data was handled and queried in the warehouse (ETL) versus the lake (ELT)? Which approach felt more adaptable to changes in data structure or format, and why?**

When working with the warehouse (ETL) approach, the data was cleaned, transformed, and structured before loading it into predefined tables. This ensured the data was consistent and ready for analysis, with a fixed schema. Queries ran smoothly because the data was already cleaned and optimized for performance.

On the other hand, with the lake (ELT) approach, the raw files were directly loaded into the data lake without modifying them. Transformations and schema were applied dynamically at query time. This was more flexible because the raw data remained as raw, allowing for adjustment of queries or transformations as needed.

Given the explanations above, the ELT approach appears to be more adaptable. Since the raw data was stored without enforcing a schema, the changes in data structure or format could be handled easily. For example, if new fields were added or the file format was changed, the transformations or queries could be simply updated without reloading the data. However, the schema needs to be defined upfront in ETL approach, which makes that approach less flexible.

**2. What challenges did you encounter when transforming and querying the data in each approach? How did these challenges help you better understand the trade-offs of schema-on-write vs. schema-on-read?**

In ETL, the data needed to be cleaned and filtered (removing rows with missing values, matching the data types defined in the tables) to match the predefined schema, otherwise it wouldn't be inserted into tables.
In ELT, the approach is more flexible but transforming the data dynamically during querying requires extra effort to handle raw files, and extra time -> performance might be affected. This may not me noticable for small datasets, but for larger ones constantly applying transformations when querying can impact the performance. 
The trade-offs are very general, if the goal is to care less about the data structure and load it as fast as possible, then querying will be less efficient and take more time. On the other hand, if we want to spend more time working on processing the data before loading it in tables, the query performance will be optimized afterwards.

**3. What factors would you consider when deciding between a warehouse, a lake, or a hybrid approach for a real-world data solution?**

A data warehouse is better for structured data, such as transactional records, or data where schema is constant and will not change in the future, where schema consistency and high-performance querying are essential. On the other hand, a data lake is better for handling unstructured and semi-structured data, such as images, videos etc. If we are dealing with a combination of both structured and unstructured data, a hybrid approach can be used.

DW is better for BI, reporting and tasks which require repeatable queries. Data lake is better for analysis, machine learning and big data operations since flexibility is needed for such cases. When both types of analysis are needed, it would be better to use the hybrid approach.

DW requires higher costs but it performs fast. On the other hand, lakes work slower but with lower costs. So depending on the trade off between cost and performance (what is the most important for the task at hand), one can choose one of the approaches.